In [ ]:
!pip install simpletransformers --quiet

In [ ]:
!pip install emoji --quiet

In [ ]:
!pip install contractions --quiet

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *

from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse

import re
import emoji
import string
import contractions

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.options.display.max_colwidth = 200

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
TRAIN_PATH = 'Train.csv'
TEST_PATH = 'Test.csv'
SAMPLE_SUB_PATH = 'SampleSubmission.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
# sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
# ID_COL = test.columns.tolist()
# TARGET_COL

In [ ]:
test.head()

,tweet_id,safe_text
0,00BHHHP1,"<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately."
1,00UNMD0E,Students starting school without whooping cough vaccinations <url> #scpick
2,01AXPTJF,"I'm kinda over every ep of <user> being ""ripped from the headlines."" Measles? Let's get back to crime. #SVU"
3,01HOEQJW,How many innocent children die for lack of vaccination each year? Around 1.5 million. Too bad all their parents couldn't be here. #SB277
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though risk is low: Federal officials said Wednesday they're taking steps… <url>"


In [ ]:
train[train['safe_text']=='1']

,tweet_id,safe_text,label,agreement
4799,I cannot believe in this day and age some parents could be so oblivious to reality as to not #vaccinate their child.,1,0.67,NaN


In [ ]:
idx = (train['safe_text']=='1')
train.loc[idx,['safe_text','label','agreement']] = train.loc[idx,['tweet_id','safe_text','label']].values
train.loc[idx,'tweet_id'] = 'MISSING'
train.loc[idx,'label'] = 1.0
train[train.index==4799]

,tweet_id,safe_text,label,agreement
4799,MISSING,I cannot believe in this day and age some parents could be so oblivious to reality as to not #vaccinate their child.,1.00,0.67


In [ ]:
train['safe_text'].apply(lambda x: len(x)).describe()

count   10001.00
mean       99.91
std        29.88
min         3.00
25%        80.00
50%       107.00
75%       122.00
max       153.00
Name: safe_text, dtype: float64

In [ ]:
train['label'].value_counts()

0.00     4908
1.00     4054
-1.00    1038
Name: label, dtype: int64

In [ ]:
train['label'][~train['label'].isin([0, -1, 1])] = -1
train.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    1
dtype: int64

In [ ]:
train = train.dropna()
train.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [ ]:
train = train[train['agreement']>0.5].reset_index(drop=True)

In [ ]:
train['label'] = train['label'].apply(int)

In [ ]:
test.head()

,tweet_id,safe_text
0,00BHHHP1,"<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately."
1,00UNMD0E,Students starting school without whooping cough vaccinations <url> #scpick
2,01AXPTJF,"I'm kinda over every ep of <user> being ""ripped from the headlines."" Measles? Let's get back to crime. #SVU"
3,01HOEQJW,How many innocent children die for lack of vaccination each year? Around 1.5 million. Too bad all their parents couldn't be here. #SB277
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though risk is low: Federal officials said Wednesday they're taking steps… <url>"


In [ ]:
test.isnull().sum()

tweet_id     0
safe_text    1
dtype: int64

In [ ]:
test['safe_text'] = test['safe_text'].fillna('xxxxxx')

test.isnull().sum()

tweet_id     0
safe_text    0
dtype: int64

In [ ]:
train.shape

(9761, 4)

In [ ]:
test.shape

(5177, 2)

## Text Clean

In [ ]:
train['ind'] = 'Train'
test['ind'] = 'Test'

In [ ]:
df=pd.concat([train,test]).reset_index(drop=True)
df.shape

(14938, 5)

In [ ]:
df.head()

,tweet_id,safe_text,label,agreement,ind
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train
1,E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1.00,1.00,Train
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1.00,1.00,Train
3,1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1.00,1.00,Train
4,J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0.00,1.00,Train


## Lower Case

In [ ]:
df['safe_text_clean'] = df['safe_text'].apply(lambda x: x.lower())

In [ ]:
df.head()

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train,me &amp; the big homie meanboy3000 #meanboy #mb #mbs #mmr #stegmanlife @ stegman st. <url>
1,E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1.00,1.00,Train,i'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the idiotic posts i've seen about world autism day
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1.00,1.00,Train,"#whatcausesautism vaccines, do not vaccinate your child"
3,1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1.00,1.00,Train,"i mean if they immunize my kid with something that won't secretly kill him years down the line then i'm all for it, but i don't trust that"
4,J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0.00,1.00,Train,thanks to <user> catch me performing at la nuit nyc 1134 1st ave. show starts at 6! #jennifair #mmr… <url>


## Remove Emoji

In [ ]:
# df['emoji_count'] = df['safe_text'].apply(lambda x: emoji.emoji_count(x))

In [ ]:
# df[df['emoji_count']>0]

In [ ]:
# Clean Emoji in text
# df['safe_text_clean'] = df['safe_text_clean'].apply(lambda x: emoji.replace_emoji(x))

In [ ]:
# # Check result
# df[df.index==69]

In [ ]:
# # Check if Emoji still in text
# df['emoji_count'] = df['safe_text_clean'].apply(lambda x: emoji.emoji_count(x))
# df[df['emoji_count']>0]

In [ ]:
# df.drop(columns=['emoji_count'], inplace=True)

In [ ]:
# df.head()

### Remove HTML tags

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [ ]:
df['safe_text_clean']=df['safe_text_clean'].apply(lambda x : remove_html(x))

In [ ]:
df[df.index==5]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
5,OVNPOAUX,<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?,1.00,0.67,Train,a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?


### Remove punctuations

In [ ]:
# Example - Before
df[df.index==0]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train,me &amp; the big homie meanboy3000 #meanboy #mb #mbs #mmr #stegmanlife @ stegman st.


In [ ]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
df['safe_text_clean'] = df['safe_text_clean'].apply(lambda x : remove_punct(x))

In [ ]:
# Example - After
df[df.index==0]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train,me amp the big homie meanboy3000 meanboy mb mbs mmr stegmanlife stegman st


### Remove Non-ASCI

In [ ]:
# Example - Before
df[df.index.isin([778,1286])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
778,LK2Z5KTX,“<user> Why you should care about the measles outbreak–even if you're vaccinated - Fortune <url> #untj4460,0.00,1.00,Train,“ why you should care about the measles outbreak–even if youre vaccinated fortune untj4460
1286,UURPI04T,Just randomly got really angry about parents not vaccinating their children ?¿?,1.00,0.67,Train,just randomly got really angry about parents not vaccinating their children ¿


In [ ]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters
    """
    return re.sub(r'[^\x00-\x7f]',r'', text)

In [ ]:
df['safe_text_clean'] = df['safe_text_clean'].apply(lambda x : remove_non_ascii(x))

In [ ]:
# Example - After
df[df.index.isin([778,1286])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
778,LK2Z5KTX,“<user> Why you should care about the measles outbreak–even if you're vaccinated - Fortune <url> #untj4460,0.00,1.00,Train,why you should care about the measles outbreakeven if youre vaccinated fortune untj4460
1286,UURPI04T,Just randomly got really angry about parents not vaccinating their children ?¿?,1.00,0.67,Train,just randomly got really angry about parents not vaccinating their children


### Expand the Contractions

In [ ]:
# Example - Before
df[df.index.isin([53,60,10672])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
53,25MG3GQ6,"They said I wasn't gone be shit, lol looked like I proved them wrong #RRR #MMR #SRT10MG We Here Baby !!!… <url>",0.00,1.00,Train,they said i wasnt gone be shit lol looked like i proved them wrong rrr mmr srt10mg we here baby
60,O44VVHXC,I don't think I will ever understand people's reasons for not getting vaccinated and not vaccinating their kids. What?!?! Explain?!?!,1.00,0.67,Train,i dont think i will ever understand peoples reasons for not getting vaccinated and not vaccinating their kids what explain
10672,6898063M,<user> I have a child with a developmental delay as well. It's up to ppl like us to make sure these kids are not getting vaccinated,NaN,NaN,Test,i have a child with a developmental delay as well its up to ppl like us to make sure these kids are not getting vaccinated


In [ ]:
df['safe_text_clean'] = df['safe_text_clean'].apply(lambda x : contractions.fix(x))

In [ ]:
# Example - After
df[df.index.isin([53,60,10672])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
53,25MG3GQ6,"They said I wasn't gone be shit, lol looked like I proved them wrong #RRR #MMR #SRT10MG We Here Baby !!!… <url>",0.00,1.00,Train,they said i was not gone be shit lol looked like i proved them wrong rrr mmr srt10mg we here baby
60,O44VVHXC,I don't think I will ever understand people's reasons for not getting vaccinated and not vaccinating their kids. What?!?! Explain?!?!,1.00,0.67,Train,i do not think i will ever understand peoples reasons for not getting vaccinated and not vaccinating their kids what explain
10672,6898063M,<user> I have a child with a developmental delay as well. It's up to ppl like us to make sure these kids are not getting vaccinated,NaN,NaN,Test,i have a child with a developmental delay as well its up to people like us to make sure these kids are not getting vaccinated


### Abbreviations

In [ ]:
# # Example - Before
# df[df.index.isin([6320,6321])]

In [ ]:
# abbreviations = {
#     "$" : " dollar ",
#     "€" : " euro ",
#     "4ao" : "for adults only",
#     "a.m" : "before midday",
#     "a3" : "anytime anywhere anyplace",
#     "aamof" : "as a matter of fact",
#     "acct" : "account",
#     "adih" : "another day in hell",
#     "afaic" : "as far as i am concerned",
#     "afaict" : "as far as i can tell",
#     "afaik" : "as far as i know",
#     "afair" : "as far as i remember",
#     "afk" : "away from keyboard",
#     "app" : "application",
#     "approx" : "approximately",
#     "apps" : "applications",
#     "asap" : "as soon as possible",
#     "asl" : "age, sex, location",
#     "atk" : "at the keyboard",
#     "ave." : "avenue",
#     "aymm" : "are you my mother",
#     "ayor" : "at your own risk",
#     "b&b" : "bed and breakfast",
#     "b+b" : "bed and breakfast",
#     "b.c" : "before christ",
#     "b2b" : "business to business",
#     "b2c" : "business to customer",
#     "b4" : "before",
#     "b4n" : "bye for now",
#     "b@u" : "back at you",
#     "bae" : "before anyone else",
#     "bak" : "back at keyboard",
#     "bbbg" : "bye bye be good",
#     "bbc" : "british broadcasting corporation",
#     "bbias" : "be back in a second",
#     "bbl" : "be back later",
#     "bbs" : "be back soon",
#     "be4" : "before",
#     "bfn" : "bye for now",
#     "blvd" : "boulevard",
#     "bout" : "about",
#     "brb" : "be right back",
#     "bros" : "brothers",
#     "brt" : "be right there",
#     "bsaaw" : "big smile and a wink",
#     "btw" : "by the way",
#     "bwl" : "bursting with laughter",
#     "c/o" : "care of",
#     "cet" : "central european time",
#     "cf" : "compare",
#     "cia" : "central intelligence agency",
#     "csl" : "can not stop laughing",
#     "cu" : "see you",
#     "cul8r" : "see you later",
#     "cv" : "curriculum vitae",
#     "cwot" : "complete waste of time",
#     "cya" : "see you",
#     "cyt" : "see you tomorrow",
#     "dae" : "does anyone else",
#     "dbmib" : "do not bother me i am busy",
#     "diy" : "do it yourself",
#     "dm" : "direct message",
#     "dwh" : "during work hours",
#     "e123" : "easy as one two three",
#     "eet" : "eastern european time",
#     "eg" : "example",
#     "embm" : "early morning business meeting",
#     "encl" : "enclosed",
#     "encl." : "enclosed",
#     "etc" : "and so on",
#     "faq" : "frequently asked questions",
#     "fawc" : "for anyone who cares",
#     "fb" : "facebook",
#     "fc" : "fingers crossed",
#     "fig" : "figure",
#     "fimh" : "forever in my heart",
#     "ft." : "feet",
#     "ft" : "featuring",
#     "ftl" : "for the loss",
#     "ftw" : "for the win",
#     "fwiw" : "for what it is worth",
#     "fyi" : "for your information",
#     "g9" : "genius",
#     "gahoy" : "get a hold of yourself",
#     "gal" : "get a life",
#     "gcse" : "general certificate of secondary education",
#     "gfn" : "gone for now",
#     "gg" : "good game",
#     "gl" : "good luck",
#     "glhf" : "good luck have fun",
#     "gmt" : "greenwich mean time",
#     "gmta" : "great minds think alike",
#     "gn" : "good night",
#     "g.o.a.t" : "greatest of all time",
#     "goat" : "greatest of all time",
#     "goi" : "get over it",
#     "gps" : "global positioning system",
#     "gr8" : "great",
#     "gratz" : "congratulations",
#     "gyal" : "girl",
#     "h&c" : "hot and cold",
#     "hp" : "horsepower",
#     "hr" : "hour",
#     "hrh" : "his royal highness",
#     "ht" : "height",
#     "ibrb" : "i will be right back",
#     "ic" : "i see",
#     "icq" : "i seek you",
#     "icymi" : "in case you missed it",
#     "idc" : "i do not care",
#     "idgadf" : "i do not give a damn fuck",
#     "idgaf" : "i do not give a fuck",
#     "idk" : "i do not know",
#     "ie" : "that is",
#     "i.e" : "that is",
#     "ifyp" : "i feel your pain",
#     "IG" : "instagram",
#     "iirc" : "if i remember correctly",
#     "ilu" : "i love you",
#     "ily" : "i love you",
#     "imho" : "in my humble opinion",
#     "imo" : "in my opinion",
#     "imu" : "i miss you",
#     "iow" : "in other words",
#     "irl" : "in real life",
#     "j4f" : "just for fun",
#     "jic" : "just in case",
#     "jk" : "just kidding",
#     "jsyk" : "just so you know",
#     "l8r" : "later",
#     "lb" : "pound",
#     "lbs" : "pounds",
#     "ldr" : "long distance relationship",
#     "lmao" : "laugh my ass off",
#     "lmfao" : "laugh my fucking ass off",
#     "lol" : "laughing out loud",
#     "ltd" : "limited",
#     "ltns" : "long time no see",
#     "m8" : "mate",
#     "mf" : "motherfucker",
#     "mfs" : "motherfuckers",
#     "mfw" : "my face when",
#     "mofo" : "motherfucker",
#     "mph" : "miles per hour",
#     "mr" : "mister",
#     "mrw" : "my reaction when",
#     "ms" : "miss",
#     "mte" : "my thoughts exactly",
#     "nagi" : "not a good idea",
#     "nbc" : "national broadcasting company",
#     "nbd" : "not big deal",
#     "nfs" : "not for sale",
#     "ngl" : "not going to lie",
#     "nhs" : "national health service",
#     "nrn" : "no reply necessary",
#     "nsfl" : "not safe for life",
#     "nsfw" : "not safe for work",
#     "nth" : "nice to have",
#     "nvr" : "never",
#     "nyc" : "new york city",
#     "oc" : "original content",
#     "og" : "original",
#     "ohp" : "overhead projector",
#     "oic" : "oh i see",
#     "omdb" : "over my dead body",
#     "omg" : "oh my god",
#     "omw" : "on my way",
#     "p.a" : "per annum",
#     "p.m" : "after midday",
#     #"pm" : "prime minister",
#     "poc" : "people of color",
#     "pov" : "point of view",
#     "pp" : "pages",
#     "ppl" : "people",
#     "prw" : "parents are watching",
#     "ps" : "postscript",
#     "pt" : "point",
#     "ptb" : "please text back",
#     "pto" : "please turn over",
#     "qpsa" : "what happens", #"que pasa",
#     "ratchet" : "rude",
#     "rbtl" : "read between the lines",
#     "rlrt" : "real life retweet",
#     "rofl" : "rolling on the floor laughing",
#     "roflol" : "rolling on the floor laughing out loud",
#     "rotflmao" : "rolling on the floor laughing my ass off",
#     "rt" : "retweet",
#     "ruok" : "are you ok",
#     "sfw" : "safe for work",
#     "sk8" : "skate",
#     "smh" : "shake my head",
#     "sq" : "square",
#     "srsly" : "seriously",
#     "ssdd" : "same stuff different day",
#     "tbh" : "to be honest",
#     "tbs" : "tablespooful",
#     "tbsp" : "tablespooful",
#     "tfw" : "that feeling when",
#     "thks" : "thank you",
#     "tho" : "though",
#     "thx" : "thank you",
#     "tia" : "thanks in advance",
#     "til" : "until",
#     "tl;dr" : "too long i did not read",
#     "tldr" : "too long i did not read",
#     "tmb" : "tweet me back",
#     "tntl" : "trying not to laugh",
#     "ttyl" : "talk to you later",
#     "u" : "you",
#     "u2" : "you too",
#     "u4e" : "yours for ever",
#     "utc" : "coordinated universal time",
#     "w/" : "with",
#     "w/o" : "without",
#     "w8" : "wait",
#     "wassup" : "what is up",
#     "wb" : "welcome back",
#     "wtf" : "what the fuck",
#     "wtg" : "way to go",
#     "wtpa" : "where the party at",
#     "wuf" : "where are you from",
#     "wuzup" : "what is up",
#     "wywh" : "wish you were here",
#     "yd" : "yard",
#     "ygtr" : "you got that right",
#     "ynk" : "you never know",
#     "zzz" : "sleeping bored and tired"
# }

In [ ]:
# abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in abbreviations.keys()) + r')(?!\w)')

In [ ]:
# def clean_abbr(text):
#     return abbr_pattern.sub(lambda x: abbreviations[x.group()], text)

In [ ]:
# df['safe_text_clean'] = df['safe_text_clean'].apply(lambda x : clean_abbr(x))

In [ ]:
# # Example - Before
# df[df.index.isin([6320,6321])]

### Replace \n

In [ ]:
# Example - Before
df[df.index.isin([132,187])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
132,4KEP2GOM,I honestly don't believe in immunization. \nNow they passed a new law that starting this year we all need specific shots to get into school.,-1.00,1.00,Train,i honestly do not believe in immunization \nnow they passed a new law that starting this year we all need specific shots to get into school
187,NH3YCR4A,"""MMR Meets Bliss"" \nThurSday AUGUST 20\n#Ladiesfree until 11 \n<user> \n<user> \n#Hot97… <url>",0.00,1.00,Train,mmr meets bliss \nthursday august 20\nladiesfree until 11 \n \n \nhot97


In [ ]:
df['safe_text_clean'] = df['safe_text_clean'].replace('\n','', regex=True)

In [ ]:
# Example - After
df[df.index.isin([132,187])]

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
132,4KEP2GOM,I honestly don't believe in immunization. \nNow they passed a new law that starting this year we all need specific shots to get into school.,-1.00,1.00,Train,i honestly do not believe in immunization now they passed a new law that starting this year we all need specific shots to get into school
187,NH3YCR4A,"""MMR Meets Bliss"" \nThurSday AUGUST 20\n#Ladiesfree until 11 \n<user> \n<user> \n#Hot97… <url>",0.00,1.00,Train,mmr meets bliss thursday august 20ladiesfree until 11 hot97


# Save Output

In [ ]:
df.head()

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train,me amp the big homie meanboy3000 meanboy mb mbs mmr stegmanlife stegman st
1,E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1.00,1.00,Train,i am 100 thinking of devoting my career to proving autism is not caused by vaccines due to the idiotic posts i have seen about world autism day
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1.00,1.00,Train,whatcausesautism vaccines do not vaccinate your child
3,1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1.00,1.00,Train,i mean if they immunize my kid with something that will not secretly kill him years down the line then i am all for it but i do not trust that
4,J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0.00,1.00,Train,thanks to catch me performing at la nuit nyc 1134 1st ave show starts at 6 jennifair mmr


In [ ]:
train = df[df['ind']=='Train'].reset_index(drop=True)
test = df[df['ind']=='Test'].reset_index(drop=True)

In [ ]:
train.shape

(9761, 6)

In [ ]:
train.head()

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0.00,1.00,Train,me amp the big homie meanboy3000 meanboy mb mbs mmr stegmanlife stegman st
1,E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1.00,1.00,Train,i am 100 thinking of devoting my career to proving autism is not caused by vaccines due to the idiotic posts i have seen about world autism day
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1.00,1.00,Train,whatcausesautism vaccines do not vaccinate your child
3,1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1.00,1.00,Train,i mean if they immunize my kid with something that will not secretly kill him years down the line then i am all for it but i do not trust that
4,J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0.00,1.00,Train,thanks to catch me performing at la nuit nyc 1134 1st ave show starts at 6 jennifair mmr


In [ ]:
test.shape

(5177, 6)

In [ ]:
test.head()

,tweet_id,safe_text,label,agreement,ind,safe_text_clean
0,00BHHHP1,"<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately.",NaN,NaN,Test,amp 4 a vaccine given 2 healthy peeps fda think just not worth the ae risk unfortunately
1,00UNMD0E,Students starting school without whooping cough vaccinations <url> #scpick,NaN,NaN,Test,students starting school without whooping cough vaccinations scpick
2,01AXPTJF,"I'm kinda over every ep of <user> being ""ripped from the headlines."" Measles? Let's get back to crime. #SVU",NaN,NaN,Test,i am kind of over every ep of being ripped from the headlines measles let us get back to crime svu
3,01HOEQJW,How many innocent children die for lack of vaccination each year? Around 1.5 million. Too bad all their parents couldn't be here. #SB277,NaN,NaN,Test,how many innocent children die for lack of vaccination each year around 15 million too bad all their parents could not be here sb277
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though risk is low: Federal officials said Wednesday they're taking steps… <url>",NaN,NaN,Test,cdc eyeing bird flu vaccine for humans though risk is low federal officials said wednesday they are taking steps


In [ ]:
train['label'] = train['label'].apply(int)

In [ ]:
clean_all= df.to_csv('./clean_all.csv',index=False)
clean_train = train.to_csv('./clean_train.csv',index=False)
clean_test = test.to_csv('./clean_test.csv',index=False)

In [ ]:
# clean_train = train.to_csv('./clean_train.csv',index=False)
# clean_test = test.to_csv('./clean_test.csv',index=False)